In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import r2_score

In [ ]:
GS = "/Users/davidhoffman/desktop/stocks_download/GS_2006-01-01_to_2018-01-01.csv"
df = pd.read_csv(GS, index_col="Date", parse_dates=True)

In [ ]:
#Only keep closing price of the day
df.drop(['Open','High','Low'], axis = 1, inplace = True)
df.drop(['Name'], axis = 1, inplace = True)
df.drop(['Volume'], axis = 1, inplace = True)

In [ ]:
train = df.Close[:-31]
test = df.Close[len(train):]
train_diff = train.diff()

In [ ]:
#ARIMA model

ar1ma1 = ARIMA(train.dropna().values, (1,1,1), freq ="D").fit()
ar1ma1_diff = ARIMA(train_diff.dropna().values, (5,1,1), freq="D").fit()

In [ ]:
#Predictions out of sample

predictions_ar1ma1 = ar1ma1.predict(start=len(train),end=len(train)+31)
full_pred_ar_ar1ma1 = train[len(train)-1]+np.cumsum(predictions_ar1ma1)

predictions_ar1ma1_diff = ar1ma1_diff.predict(start=len(train),end=len(train)+31)
full_pred_ar1ma1_diff = train[len(train)-1]+np.cumsum(predictions_ar1ma1_diff)


In [ ]:
#Actual close price vs full_pred_ar1ma1_diff, out of sample

fig, ax = plt.subplots(figsize=(12,5))
date_ticks = test.index
ax.plot(date_ticks, full_pred_ar1ma1_diff[2:], lw=2, color='grey', ls='dashed', label = 'Predictions')
ax.plot(date_ticks, test, lw=2, color='darkred', label = 'Actual')
plt.xlabel("Date")
plt.ylabel("Closing price")
plt.title("Close vs predicted close")
plt.legend()
plt.show();

In [ ]:
ar1ma1 = ARIMA(train.dropna().values, (13,2,1), freq="D").fit()

In [ ]:
len(train)

In [ ]:
ar1ma1.plot_predict(2970,2999,dynamic=False)

In [76]:
#Function to create a model and evaluate its predictions
#Arima model,
#input will be parameter p and df/col

def price_preds_p(df,days, p, column = 'Close'):
    #df = dataframe
    #column = Close --> predicting closing price
    #days --> how many days do you want to predict
    #p --> paramater p --> lags
    
    #Seperate into train and test
    train = df.Close[:-days]
    test = df.Close[len(train):]
    
    
    #Difference the train set
    #train_diff = train.diff()
    
    #fit model
    ar1ma1 = ARIMA(train.values, (p,2,1)).fit(disp=0)
    
    #predict out of sample ---> test set
    preds = ar1ma1.predict(start=len(train), end=len(train)+days-1)

    #Add predictions to the last value in train set
    full_preds = train[len(train)-1] + np.cumsum(preds)
   
    #for i in range(days):
        #print ("actual", test[i], "predicted", full_preds[i])
        #print ("difference (pred - actual)", full_preds[i]-test[i])
    return np.var(full_preds)

results for price preds p function:
p=:
1.    0.004345001535578772

2.    0.0034985237621212094

3.    0.00452138116930596

4.    0.0073039167950536025

5.    0.014790984047964974

6.    0.009266575375453334

7.    0.010224646837947162

8.    0.027918079364850162

9.    0.04855250276397046

10.   0.04927565725958586

11.   0.04732463398094426

12.   0.04745722050024483

13.   0.05185252215646854

14.   0.048293020780350894

15.   0.04047066591073391

16.   0.030693392841559006







In [87]:
"""for i in range(1,20):
    x = price_preds_p(df=df,days=1, p= i, column = 'Close')
    print (i," .  ", x)"""

'for i in range(1,20):\n    x = price_preds_p(df=df,days=1, p= i, column = \'Close\')\n    print (i," .  ", x)'